In [4]:
import pytz

from zipline.algorithm import TradingAlgorithm
from zipline.utils.factory import load_from_yahoo
from zipline.finance import commission

zipline_logging = logbook.NestedSetup([
    logbook.NullHandler(level=logbook.DEBUG),
    logbook.StreamHandler(sys.stdout, level=logbook.INFO),
    logbook.StreamHandler(sys.stderr, level=logbook.ERROR),
])
zipline_logging.push_application()


In [106]:
import pandas as pd
t1=pd.to_datetime("2000-1-1",format="%Y-%m-%d")
t2=pd.to_datetime("2000-1-2",format="%Y-%m-%d")
d = {'stock_symbol': ('BANANA','AAPL'), 'stock_date': (t1,t2)}
df = pd.DataFrame(data=d, index={0,1})
df.stock_date[df.stock_symbol=='AAPL']
df.ix[0].stock_date
len(df)
type(df.ix[1].stock_symbol)

str

In [7]:
%load_ext zipline

In [131]:
%%zipline --start 2000-1-1 --end 2002-1-1

import pandas as pd

from zipline.api import symbol, order, record

actual_time=0
def initialize(context):
    context.count=0
    t1=pd.to_datetime("2000-1-5",format="%Y-%m-%d")
    t2=pd.to_datetime("2000-1-7",format="%Y-%m-%d")
    d = {'stock_symbol': ('XOM','AAPL'), 'stock_date': (t1,t2),'quantity':(10,-20)}
    df = pd.DataFrame(data=d, index={0,1})
#     df.stock_date[df.stock_symbol=='AAPL']
#     df.ix[0].stock_date
    context.df=df
    print()

def handle_data(context, data):
    symb=context.df.ix[context.count].stock_symbol
    if(context.count<len(context.df)):
        actual_time = data.current(symbol(symb), "last_traded")
        actual_time =actual_time.tz_localize(None)
        if (context.df.ix[context.count].stock_date==actual_time):
            print(context.count)
            order(symbol(symb), int(context.df.ix[context.count].quantity))
            record(Count=context.count,Date=context.df.ix[context.count].stock_date,Security_Price=data[symbol(symb)].price)
            if(context.count<len(context.df)-1):
                context.count+=1
    


0
1


,Count,Date,Security_Price,algo_volatility,algorithm_period_return,alpha,benchmark_period_return,benchmark_volatility,beta,capital_used,...,short_exposure,short_value,shorts_count,sortino,starting_cash,starting_exposure,starting_value,trading_days,transactions,treasury_period_return
2000-01-03 21:00:00+00:00,NaN,NaT,NaN,NaN,0.000000e+00,NaN,-0.009549,NaN,NaN,0.0,...,0.0,0.0,0,NaN,10000000.0,0.0,0.0,1,[],0.0658
2000-01-04 21:00:00+00:00,NaN,NaT,NaN,0.000000e+00,0.000000e+00,0.000000,-0.047528,0.323229,0.000000,0.0,...,0.0,0.0,0,NaN,10000000.0,0.0,0.0,2,[],0.0649
2000-01-05 21:00:00+00:00,0.0,2000-01-05,80.63,0.000000e+00,0.000000e+00,0.000000,-0.045697,0.329321,0.000000,0.0,...,0.0,0.0,0,NaN,10000000.0,0.0,0.0,3,[],0.0662
2000-01-06 21:00:00+00:00,0.0,2000-01-05,80.63,7.937254e-07,-1.000000e-07,-0.000010,-0.044785,0.298325,-0.000001,-852.9,...,0.0,0.0,0,-7.937254,10000000.0,0.0,0.0,4,[{'order_id': '1f8c3a7846c74b08a43293f3de4e5c5...,0.0657
2000-01-07 21:00:00+00:00,1.0,2000-01-07,99.50,1.738965e-06,-3.500000e-07,-0.000021,-0.018908,0.375301,-0.000004,0.0,...,0.0,0.0,0,-9.228143,9999147.1,851.9,851.9,5,[],0.0652
2000-01-10 21:00:00+00:00,1.0,2000-01-07,99.50,8.058042e-06,-1.640000e-06,-0.000072,-0.007929,0.349070,-0.000010,1954.0,...,-1955.0,-1955.0,1,-8.065264,9999147.1,849.4,849.4,6,[{'order_id': 'fa89f57c37f94e51b15c3312eb29402...,0.0657
2000-01-11 21:00:00+00:00,1.0,2000-01-07,99.50,6.803892e-05,9.360000e-06,0.000298,-0.020888,0.326609,-0.000055,0.0,...,-1855.0,-1855.0,1,42.616509,10001101.1,-1117.5,-1117.5,7,[],0.0667
2000-01-12 21:00:00+00:00,1.0,2000-01-07,99.50,7.912913e-05,1.923000e-05,0.000561,-0.025183,0.302508,-0.000059,0.0,...,-1743.8,-1743.8,1,81.899797,10001101.1,-1007.5,-1007.5,8,[],0.0672
2000-01-13 21:00:00+00:00,1.0,2000-01-07,99.50,1.273461e-04,2.050000e-06,0.000007,-0.013320,0.294168,-0.000151,0.0,...,-1935.0,-1935.0,1,0.629649,10001101.1,-908.8,-908.8,9,[],0.0663
2000-01-14 21:00:00+00:00,1.0,2000-01-07,99.50,1.288163e-04,-7.020000e-06,-0.000183,-0.002791,0.283818,-0.000179,0.0,...,-2008.8,-2008.8,1,-1.809753,10001101.1,-1080.6,-1080.6,10,[],0.0669


In [23]:
type(actual_time)

NameError: name 'actual_time' is not defined

In [9]:
_.head()

,AAPL,algo_volatility,algorithm_period_return,alpha,benchmark_period_return,benchmark_volatility,beta,capital_used,ending_cash,ending_exposure,...,short_exposure,short_value,shorts_count,sortino,starting_cash,starting_exposure,starting_value,trading_days,transactions,treasury_period_return
2000-01-03 21:00:00+00:00,111.94,NaN,0.000000e+00,NaN,-0.009549,NaN,NaN,0.0,10000000.0,0.0,...,0,0,0,NaN,10000000.0,0.0,0.0,1,[],0.0658
2000-01-04 21:00:00+00:00,102.50,0.000001,-1.000000e-07,0.000008,-0.047528,0.323229,0.000003,-1026.0,9998974.0,1025.0,...,0,0,0,-11.224972,10000000.0,0.0,0.0,2,[{'order_id': '108b19a7c5c040eea9bbf18fc3e1ddc...,0.0649
2000-01-05 21:00:00+00:00,104.00,0.000013,1.300000e-06,0.000228,-0.045697,0.329321,0.000031,-1041.0,9997933.0,2080.0,...,0,0,0,119.146981,9998974.0,1025.0,1025.0,3,[{'order_id': '5eff4550b49d4f45b6074ac37cd0491...,0.0662
2000-01-06 21:00:00+00:00,95.00,0.000148,-1.680000e-05,-0.001593,-0.044785,0.298325,-0.000189,-951.0,9996982.0,2850.0,...,0,0,0,-7.367062,9997933.0,2080.0,2080.0,4,[{'order_id': '4ffe16eb6dcd4a949f796086d08b17d...,0.0657
2000-01-07 21:00:00+00:00,99.50,0.000179,-3.400000e-06,-0.000034,-0.018908,0.375301,0.000152,-996.0,9995986.0,3980.0,...,0,0,0,-1.333453,9996982.0,2850.0,2850.0,5,[{'order_id': '823272abe2bf414794adfda74f67e02...,0.0652
